In [1]:
import os
os.environ["MLFLOW_TRACKING_URI"] = "http://127.0.0.1:5000"


In [2]:
# Import des librairies nécessaires
import mlflow
import mlflow.sklearn
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pandas as pd

# Chargement du jeu de données Iris pour l'exemple
app_train = pd.read_csv('Base De Donnée Prétraitée.csv')
app_test = pd.read_csv('Base De Donnée Prétraitée Test.csv')

In [3]:
app_test

,Unnamed: 0,SK_ID_CURR,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes,DAYS_EMPLOYED_ANOM,CREDIT_INCOME_PERCENT,ANNUITY_INCOME_PERCENT,CREDIT_TERM,DAYS_EMPLOYED_PERCENT
0,0,100001,0,0,1,0,135000.0,568800.0,20560.5,450000.0,...,False,True,False,True,False,False,4.213333,0.152300,0.036147,0.121044
1,1,100005,0,0,1,0,99000.0,222768.0,17370.0,180000.0,...,False,False,False,False,False,False,2.250182,0.175455,0.077973,0.247398
2,2,100013,0,1,1,0,202500.0,663264.0,69777.0,630000.0,...,False,False,False,False,False,False,3.275378,0.344578,0.105202,0.222477
3,3,100028,0,0,1,2,315000.0,1575000.0,49018.5,1575000.0,...,True,False,False,True,False,False,5.000000,0.155614,0.031123,0.133515
4,4,100038,0,1,0,1,180000.0,625500.0,32067.0,625500.0,...,False,False,False,False,False,False,3.475000,0.178150,0.051266,0.168021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48739,48739,456221,0,0,1,0,121500.0,412560.0,17473.5,270000.0,...,False,False,False,False,False,False,3.395556,0.143815,0.042354,0.258838
48740,48740,456222,0,0,0,2,157500.0,622413.0,31909.5,495000.0,...,False,False,False,False,False,False,3.951829,0.202600,0.051267,0.102718
48741,48741,456223,0,1,1,1,202500.0,315000.0,33205.5,315000.0,...,False,True,False,True,False,False,1.555556,0.163978,0.105414,0.190742
48742,48742,456224,0,0,0,0,225000.0,450000.0,25128.0,450000.0,...,True,False,False,True,False,False,2.000000,0.111680,0.055840,0.195518


In [25]:
X = data.drop(columns="TARGET", axis=1)
y = data["TARGET"]

In [26]:
# Séparation des données en ensemble d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Regression Logistique 

In [28]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer

# Drop the target from the training data
if 'TARGET' in app_train:
    train = app_train.drop(columns = ['TARGET'])
else:
    train = app_train.copy()
    
# Feature names
features = list(train.columns)

# Copy of the testing data
test = app_test.copy()

# Median imputation of missing values
imputer = SimpleImputer(strategy = 'median')

# Scale each feature to 0-1
scaler = MinMaxScaler(feature_range = (0, 1))

# Fit on the training data
imputer.fit(train)

# Transform both training and testing data
train = imputer.transform(train)
test = imputer.transform(app_test)

# Repeat with the scaler
scaler.fit(train)
train = scaler.transform(train)
test = scaler.transform(test)

print('Training data shape: ', train.shape)
print('Testing data shape: ', test.shape)

Training data shape:  (307511, 241)
Testing data shape:  (48744, 241)


In [ ]:
# Séparation des données en ensemble d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(train, app_train["TARGET"], test_size=0.2, random_state=0)

In [ ]:
from sklearn.linear_model import LogisticRegression

# Make the model with the specified regularization parameter
log_reg = LogisticRegression(C = 0.0001)

# Train on the training data
log_reg.fit(train, app_train["TARGET"])

In [14]:
# Définir les paramètres du modèle
n_estimators = 50
max_depth = 5
random_state = 42

In [15]:
# Début d'une nouvelle expérimentation MLFlow
with mlflow.start_run():
    # Initialisation et entraînement du modèle RandomForest
    model = LogisticRegression(C = 0.0001)
    model.fit(X_train, y_train)

    # Prédictions et calcul de la précision
    predictions = model.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)

    # Loguer les paramètres et les métriques dans MLFlow
    mlflow.log_param("n_estimators", n_estimators)
    mlflow.log_param("max_depth", max_depth)
    mlflow.log_param("random_state", random_state)
    mlflow.log_metric("accuracy", accuracy)

    # Loguer le modèle dans MLFlow
    mlflow.sklearn.log_model(model, "random_forest_model")

    print(f"Précision du modèle : {accuracy}")
    print("Modèle et métriques enregistrés avec succès dans MLFlow")

# Conseil : Lancez mlflow ui dans un terminal pour voir les résultats en cours

2024/10/11 12:45:22 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/10/11 12:45:22 INFO mlflow.tracking._tracking_service.client: 🏃 View run monumental-moose-611 at: http://127.0.0.1:5000/#/experiments/0/runs/59bbbd8da13b47c5a0a67885b1351a55.
2024/10/11 12:45:22 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/0.


Précision du modèle : 0.9210607612636782
Modèle et métriques enregistrés avec succès dans MLFlow
